In [6]:
!pip install -U albumentations

In [5]:
!pip install --upgrade paramiko cryptography


In [7]:
!pip install numpy

In [11]:
import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET
import albumentations as A

OSError: Error loading libscipy_openblas64_-caad452230ae4ddb57899b8b3a33c55c.dll; %1 is not a valid Win32 application.

In [ ]:
#Path
image_dir = 'Data_Set_Larch_Casebearer/Imagedata'
annotation_dir  = 'Data_Set_Larch_Casebearer/Annotations'
save_dir = 'Data_Set_Larch_Casebearer/Croppedimages'
output_annotation_dir = 'Data_Set_Larch_Casebearer/Augmented_Annotations'


In [18]:
def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    bboxes = []
    labels = []

   
    for obj in root.findall('object'):
        # Check if the 'damage' tag exists
        damage_tag = obj.find('damage')
        if damage_tag is not None:
            label = damage_tag.text  # Healthy, Light Damage, High Damage, Others
        else:
            label = 'unknown'  # Handle the case where 'damage' tag is missing

        # Parse bounding box coordinates
        bndbox = obj.find('bndbox')
        if bndbox is not None:
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            bboxes.append([xmin, ymin, xmax, ymax])
            labels.append(label)

    return bboxes, labels

In [19]:
import os
import cv2

# Function to crop the images based on bounding boxes and save them
def crop_and_save_images(image_dir, annotation_dir, save_dir):
    # Iterate over all images and corresponding annotations
    for img_file in os.listdir(image_dir):
        if img_file.endswith('.JPG') or img_file.endswith('.png'):
            img_path = os.path.join(image_dir, img_file)
            
            # Corresponding XML annotation file
            xml_file = os.path.join(annotation_dir, os.path.splitext(img_file)[0] + '.xml')
            
            # Check if the XML file exists
            if os.path.exists(xml_file):
                # Read image
                img = cv2.imread(img_path)
                
                # Check if the image is loaded correctly
                if img is None:
                    print(f"Error: Unable to load image {img_path}")
                    continue  # Skip this image and continue with the next one

                # Parse the XML file to get bounding boxes and labels
                bboxes, labels = parse_xml(xml_file)

                height, width, _ = img.shape
                
                # Crop and save the trees
                for i, bbox in enumerate(bboxes):
                    xmin, ymin, xmax, ymax = bbox

                    # Validate bounding boxes (make sure they are within the image boundaries)
                    if xmin < 0 or ymin < 0 or xmax > width or ymax > height:
                        print(f"Warning: Bounding box {bbox} is out of bounds for image {img_file}")
                        continue  # Skip this bounding box

                    # Crop the region defined by the bounding box
                    cropped_img = img[ymin:ymax, xmin:xmax]

                    # Ensure the cropped image is not empty
                    if cropped_img.size > 0:
                        label = labels[i]

                        # Ensure save directory exists
                        if not os.path.exists(save_dir):
                            os.makedirs(save_dir)

                        # Save cropped image based on label
                        cropped_save_path = os.path.join(save_dir, f'{label}_{os.path.splitext(img_file)[0]}_{i}.jpg')
                        cv2.imwrite(cropped_save_path, cropped_img)
                    else:
                        continue
            else:
                continue


In [20]:
crop_and_save_images(image_dir, annotation_dir , save_dir)
print("Images are cropped!!")

Images are cropped!!


In [34]:
image_list = []
label_list = []

# Dictionary to map filename prefix to class label
label_map = {
    'H': 0,   # Healthy
    'LD': 1,  # Lightly Damaged
    'HD': 2,  # Heavily Damaged
    'other': 3    # Others
}


In [35]:
for filename in os.listdir(save_dir):
    # Only process image files (you can modify the extension as needed)
    if filename.endswith('.JPG') or filename.endswith('.jpg'):
        # Extract the label from the filename (first part of the filename like 'H', 'HD', etc.)
        label_prefix = filename.split('_')[0]
        
        # Append the full path of the image to image_list
        image_list.append(os.path.join(save_dir, filename))
        
        # Append the corresponding label to label_list using label_map
        if label_prefix in label_map:
            label_list.append(label_map[label_prefix])
        else:
            print(f"Warning: Unknown label prefix {label_prefix} in file {filename}")

In [36]:
print(image_list[0:10])
print(label_list[0:10])
print(len(image_list))
print(len(label_list))

['Data_Set_Larch_Casebearer/Croppedimages\\HD_B01_0004_14.jpg', 'Data_Set_Larch_Casebearer/Croppedimages\\HD_B01_0004_15.jpg', 'Data_Set_Larch_Casebearer/Croppedimages\\HD_B01_0004_16.jpg', 'Data_Set_Larch_Casebearer/Croppedimages\\HD_B01_0004_17.jpg', 'Data_Set_Larch_Casebearer/Croppedimages\\HD_B01_0004_18.jpg', 'Data_Set_Larch_Casebearer/Croppedimages\\HD_B01_0004_20.jpg', 'Data_Set_Larch_Casebearer/Croppedimages\\HD_B01_0004_21.jpg', 'Data_Set_Larch_Casebearer/Croppedimages\\HD_B01_0004_22.jpg', 'Data_Set_Larch_Casebearer/Croppedimages\\HD_B01_0004_23.jpg', 'Data_Set_Larch_Casebearer/Croppedimages\\HD_B01_0004_24.jpg']
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
101831
101831


In [38]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets (80% training, 20% validation)
train_images, val_images, train_labels, val_labels = train_test_split(
    image_list, label_list, test_size=0.3, random_state=42, stratify=label_list
)

# Verify the split
print(f"Training set size: {len(train_images)}")
print(f"Validation set size: {len(val_images)}")

Training set size: 71281
Validation set size: 30550


In [40]:
!pip install transformers tensorflow


  Using cached transformers-4.45.2-py3-none-any.whl (9.9 MB)
  Using cached tokenizers-0.20.1-cp39-none-win_amd64.whl (2.4 MB)


In [44]:
from transformers import AutoFeatureExtractor, TFAutoModelForImageClassification
import tensorflow as tf

# Load the feature extractor (for preprocessing) and pre-trained Swin Transformer model
extractor = AutoFeatureExtractor.from_pretrained('microsoft/swin-tiny-patch4-window7-224')
model = TFAutoModelForImageClassification.from_pretrained('microsoft/swin-tiny-patch4-window7-224', num_labels=4)


AttributeError: module 'lib' has no attribute 'X509_V_FLAG_NOTIFY_POLICY'

In [20]:
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Custom data generator class
class TreeDataGenerator(Sequence):
    def __init__(self, image_list, label_list, batch_size, target_size, shuffle=True):
        self.image_list = image_list
        self.label_list = label_list
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.image_list))
        self.on_epoch_end()

    def __len__(self):
        # Returns the number of batches per epoch
        return len(self.image_list) // self.batch_size

    def __getitem__(self, index):
        # Generate indexes for this batch
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        
        # Get the batch of images and labels
        batch_image_list = [self.image_list[i] for i in batch_indexes]
        batch_label_list = [self.label_list[i] for i in batch_indexes]

        # Load images and resize them
        images = [img_to_array(load_img(img_path, target_size=self.target_size)) / 255.0 for img_path in batch_image_list]
        
        # Convert lists to numpy arrays
        X = np.array(images)
        y = np.array(batch_label_list)
        
        return X, y

    def on_epoch_end(self):
        # Shuffle the data at the end of each epoch
        if self.shuffle:
            np.random.shuffle(self.indexes)

# Create an instance of the data generator
train_gen = TreeDataGenerator(image_list, label_list, batch_size=32, target_size=(250, 250))


In [21]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets (80% training, 20% validation)
train_images, val_images, train_labels, val_labels = train_test_split(
    image_list, label_list, test_size=0.2, random_state=42, stratify=label_list
)

ValueError: Your 'label_fields' are not valid - them must have same names as params in dict